In [46]:
!pip install hazm faiss-cpu tqdm

## Import libraries

In [36]:
import pandas as pd
from hazm import Normalizer, WordTokenizer, Stemmer, Lemmatizer, stopwords_list
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import faiss
import numpy as np
from tqdm import tqdm
import json
from google.colab import files
import time

## Upload dataset:
I used BBC Persian News dataset

In [3]:
uploaded = files.upload()
filename = list(uploaded.keys())[0]

Saving bbcpersian.json to bbcpersian.json


## Open dataset file and extract Titles and Bodies

In [34]:
with open(filename, 'r', encoding='utf-8') as f:
    data = json.load(f)

# extract Titles and Bodies
titles = [item['title'] for item in data]
bodies = [item['body'] for item in data]


# example of body
for body in bodies[:5]:
    print(body)

برای پخش این فایل لطفا جاوا اسکریپت را فعال یا از یک مرورگر دیگر استفاده کنید.
تهران و دوبی؛ ملاقات مجازی، منافع حقیقی

وزرای خارجه ایران و امارات روز یکشنبه به شکل مجازی با هم ملاقات کردند. امارات از سال ۹۴ و حمله به سفارت عربستان در تهران، سطح روابطش را با ایران کاهش داده، اما از آغاز بحران کرونا، تنش میان دو کشور کاهش یافته است. امارات یکی از نخستین کشورهایی بود که در آغاز بحران کرونا به ایران کمک کرد.
کیوان حسینی گزارش می‌دهد.




یک نظرسنجی تازه جهانی نشان دهنده عمق اضطرابی است که بسیاری جوانان درباره تغییر اقلیم احساس می کنند.
تقریبا ۶۰ درصد جوانانی که در این بررسی مورد پرسش قرار گرفتند گفتند که خیلی یا شدیدا احساس نگرانی می کنند.
بیش از ۴۵ درصد پرسش شوندگان گفتند احساسات آنها درباره اقلیم بر زندگی روزمره آنها اثر می گذارد.
سه چهارم گفتند که فکر می کنند آینده ترسناک است. بیش از نیمی (۵۶ درصد) گفتند به نظر آنها سرنوشت بشر محتوم است.

دو سوم گفتند که احساس اندوه، ترس و اضطراب می کنند. بسیاری احساس بیم، خشم، عجز، اندوه و شرم - و همچنین امید - می کردند.
یک جوان ۱۶ ساله گفت: "برای جوا

## Define normalization, tokenization, stemming, lemmatization and stop words

In [5]:
normalizer = Normalizer()
tokenizer = WordTokenizer()
stemmer = Stemmer()
lemmatizer = Lemmatizer()
stop_words = set(stopwords_list())

## Preprocessing function
normalization, tokenization, stemming, and lemmatization

In [6]:
def preprocess(text):
    text = normalizer.normalize(text)
    tokens = tokenizer.tokenize(text)
    tokens = [t for t in tokens if t not in stop_words]
    tokens = [lemmatizer.lemmatize(stemmer.stem(t)) for t in tokens]
    return " ".join(tokens)

## Preprocess, TF-vectorization and FAISS indexing for FAST Similarity search

In [7]:
documents = [preprocess(text) for text in tqdm(bodies)]

# TF-IDF vectorization
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(documents)
# FAISS Indexing for Fast Search
faiss_index = faiss.IndexFlatIP(tfidf_matrix.shape[1])
faiss_index.add(tfidf_matrix.toarray())

100%|██████████| 3780/3780 [00:40<00:00, 93.71it/s] 


## Normal search

In [37]:
def normal_search(query, top_k=5):
    query_vector = vectorizer.transform([preprocess(query)])
    similarities = cosine_similarity(query_vector, tfidf_matrix).flatten()
    top_indices = similarities.argsort()[-top_k:][::-1]
    return [(titles[i], bodies[i], similarities[i]) for i in top_indices]

## Fast search function: uses vectorization & indexing

In [38]:
def fast_search(query, top_k=5):
    # vectorize the query after preprocessing
    query_vector = vectorizer.transform([preprocess(query)]).toarray()
    # search the FAISS index
    scores, indices = faiss_index.search(query_vector, top_k)
    return [(titles[i], bodies[i], scores[0][j]) for j, i in enumerate(indices[0])]

## An example for IR

In [9]:
results = fast_search("اقتصاد ایران", top_k=5)

for title, body, score in results:
    print(f"Title: {title}\nScore: {score:.4f}\nBody: {body[:200]}...\n{'='*50}\n")


Title: تغییر دولت‌ها در ایران و سرگردانی و تعلیق برای فعالان اقتصادی
Score: 0.5072
Body: 


مقاطع زمانی تغییر دولت‌ها در ایران برای فعالان اقتصادی‌ به معنای سرگردانی و تعلیق است، اینکه باید مدتی را دست نگاه دارند تا ببینند دولت جدید قرار است چه بر سر اقتصاد ایران بیاورد و میانه‌ دولتمردان...

Title: تناقض آمارها درباره رشد اقتصادی ایران؛ روحانی رشد اقتصادی سال پیش بدون نفت را 'مثبت' اعلام کرد
Score: 0.5045
Body: 


در حالیکه رئیس بانک مرکزی و مرکز آمار ایران اخیرا داده‌های متناقضی درباره رشد اقتصادی ایران در سال گذشته ( ۹۸ ) منتشر کرده‌اند، حسن روحانی رئیس‌جمهوری ایران، آمارهای ارائه شده  توسط بانک مرکزی را ت...

Title: پیش‌بینی بانک جهانی درباره رکود در دنیا؛ در سال ۲۰۲۰ 'اقتصاد ایران ۵/۳ درصد کوچکتر می‌شود'
Score: 0.4840
Body: 


بانک جهانی در گزارشی که درباره چشم‌انداز اقتصاد جهان در سال جاری میلادی (۲۰۲۰) منتشر کرده به تفصیل به تاثیر منفی شیوع کووید-۱۹ بر اقتصاد جهان و کشورهای مختلف پرداخته است.
در این گزارش به سرعت "هشدا...

Title: پایان قرن؛ اقتصاد ایران در صد سالی که گذشت
Score:

## Another example for IR

In [10]:
results = fast_search("دلار", top_k=5)

for title, body, score in results:
    print(f"Title: {title}\nScore: {score:.4f}\nBody: {body[:200]}...\n{'='*50}\n")

Title: رئیس بانک مرکزی ایران در عراق در طلب بدهی‌های انباشتی میلیارد دلاری
Score: 0.3813
Body: برای پخش این فایل لطفا جاوا اسکریپت را فعال یا از یک مرورگر دیگر استفاده کنید.
رئیس بانک مرکزی ایران در عراق در طلب بدهی‌های انباشتی میلیارد دلاری

بانک مرکزی ایران اعلام کرده که از امروز برای جلوگیری...

Title: رکوردهای تازه قیمت دلار در ایران؛ انگشت اتهام مجلس به سوی دولت
Score: 0.3350
Body: برای پخش این فایل لطفا جاوا اسکریپت را فعال یا از یک مرورگر دیگر استفاده کنید.
رکوردهای تازه قیمت دلار در ایران؛ انگشت اتهام مجلس به سوی دولت

قیمت دلار در بازار ایران به رقم بی‌سابقه ۲۲ هزار تومان رس...

Title: آیا دلار به '۱۵ هزار تومان' برمی‌گردد؟
Score: 0.3153
Body: 
هادی چاوشیروزنامه‌نگار


سخنان رئیس‌جمهور ایران درباره احتمال رسیدن قیمت دلار به ۱۵ هزار تومان، همزمان با کاهش قیمت ارزهای معتبر در بازار تهران طی روزهای اخیر، این سوال را برای بسیاری از شهروندان پیش...

Title: بیت‌کوین قیمت بالای '۵۰ هزار دلار' را هم تجربه کرد
Score: 0.2887
Body: 


براساس گزارش رویترز قیمت بیت‌کوین در معاملات امروز سه

## Comparing query time

In [45]:
query = "اقتصاد ایران"

start = time.time()
_ = fast_search(query, top_k=5)
print(f"fast_search duration: {time.time() - start:.4f} seconds")

start = time.time()
_ = normal_search(query, top_k=5)
print(f"normal_search duration: {time.time() - start:.4f} seconds")

fast_search duration: 0.0657 seconds
normal_search duration: 0.0223 seconds
